In [3]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import datetime
import time
import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')


from IPython.display import display

#Moving Average  
def MA(df, n):  
    MA = df.rolling(n).mean()
    MA.columns = df.columns
    return MA

#Bollinger Bands  
def BBANDS(df, n, multiple):  
    MA = df.rolling(n).mean() 
    MSD = df.rolling(n).std()
    ub1 =  MA + multiple * MSD
    uB1 = pd.DataFrame(ub1, index = df.index, columns = df.columns)  
    lb1 =  MA - multiple * MSD
    lB1 = pd.DataFrame(lb1, index = df.index, columns = df.columns)  
    b2 = (df - MA + multiple * MSD) / (2 * multiple * MSD)  
    B2 = pd.DataFrame(b2, index = df.index, columns = df.columns)  
    BBAND = pd.concat([MA, uB1, lB1, B2], keys = ['Mean', 'UpperBand', 'LowerBand', '%B'], axis = 1)
    return BBAND

#Average True Range  
def ATR(df, n, H_price_name='High', L_price_name='Low', C_price_name='Close'):  
    i = 0  
    TR_l = [0]  
    while i < len(df.index)-1:  
        TR = max(df[H_price_name][i+1], df[C_price_name][i]) - min(df[L_price_name][i+1], df[C_price_name][i])  
        TR_l.append(TR)  
        i = i + 1  
    TR_s = pd.Series(TR_l,index = df.index)  
    ATR = pd.DataFrame(TR_s.ewm(span = n, min_periods = n).mean(), columns = ['ATR_%s' % n])
    return ATR

#Momentum  
def MOM(df, n, C_price_name='Close'):  
    M = pd.Series(df[C_price_name].diff(n), name = 'Momentum_' + str(n))    
    return M

#Keltner Channel  
def KELCH(df, n, std, H_price_name='High', L_price_name='Low', C_price_name='Close'):  
    KelChM = pd.Series(df[C_price_name].rolling(n).mean())
    ATR_value = ATR(df, n, H_price_name, L_price_name, C_price_name)['ATR_%s' % n]
    KelChU = KelChM + std * ATR_value
    KelChD = KelChM - std * ATR_value

    KEL = pd.concat([KelChU, KelChM, KelChD], keys = ['KelChU', 'KelChM', 'KelChD'], axis = 1) 
    return KEL

#Keltner Channel  
def SQUEEZE(df, b_n, b_std, k_n, k_std, H_price_name='High', L_price_name='Low', C_price_name='Close'):  
    
    BBAND = BBANDS(df[[C_price_name]], b_n, b_std)
    BBAND.columns = BBAND.columns.get_level_values(0)
    UB_B = BBAND['UpperBand']
    LB_B = BBAND['LowerBand']
    
    KEL = KELCH(df, k_n, k_std, H_price_name, L_price_name, C_price_name)
    UB_K = KEL['KelChU']
    LB_K = KEL['KelChD']
    
    check = pd.concat([BBAND, KEL], axis = 1)
  
    Signal = pd.DataFrame(((UB_B < UB_K) & (LB_B > LB_K)).astype(int), columns = ['squeeze'])
    Signal['Momentum'] = MOM(df, k_n, C_price_name)

    return Signal, check

#Relative Strength Index  
def RSI(df, n, H_price_name='High', L_price_name='Low'):   
    i = 0  
    UpI = [0]  
    DoI = [0]  
    while i + 1 <= len(df.index)-1:  
        UpMove = df[H_price_name][i + 1] - df[H_price_name][i]  
        DoMove = df[L_price_name][i] - df[L_price_name][i+1]
        if UpMove > DoMove and UpMove > 0:  
            UpD = UpMove  
        else: UpD = 0  
        UpI.append(UpD)  
        if DoMove > UpMove and DoMove > 0:  
            DoD = DoMove  
        else: DoD = 0  
        DoI.append(DoD) 
        i = i + 1  
    UpI = pd.DataFrame(UpI,index = df.index)  
    DoI = pd.DataFrame(DoI, index = df.index)  
    PosDI = pd.DataFrame(UpI.ewm(span = n, min_periods = n - 1).mean())  
    NegDI = pd.DataFrame(DoI.ewm(span = n, min_periods = n - 1).mean())  
    RSI = 100.0 - 100.0 / (1 + (PosDI /NegDI))
    RSI.columns = ['RSI']
    return RSI

#MACD, MACD Signal and MACD difference  
def MACD(df, n_fast, n_slow, C_price_name='Close'):   
    EMAfast = df[C_price_name].ewm(span = n_fast, min_periods = n_slow - 1).mean() 
    EMAslow = df[C_price_name].ewm(span = n_slow, min_periods = n_slow - 1).mean() 
    MACD = pd.Series(EMAfast - EMAslow, name = 'MACD_' + str(n_fast) + '_' + str(n_slow))  
    MACDsign = pd.Series(MACD.ewm(span = 9, min_periods = 8).mean(), name = 'MACDsign_' + str(n_fast) + '_' + str(n_slow))  
    MACDdiff = pd.Series(MACD - MACDsign, name = 'MACDdiff_' + str(n_fast) + '_' + str(n_slow))  
    MACD = pd.concat([MACD, MACDsign, MACDdiff], axis = 1)

    return MACD

def EMA(df, n):  
    EMA = df.ewm(span = n, min_periods = n - 1).mean()
    EMA.columns = ['%s_EMA_%s' % (i, n) for i in df.columns]
    return EMA

def performance_matrics(port_value, output = False):

    #annual volatility
    ann_vol = (port_value.pct_change().std()) * np.sqrt(52)
    port_return = pd.DataFrame(index = port_value.index[:], columns=['Values'])
    port_return['Values'] = port_value.pct_change()

    #cumulative return series & max drawdown, max drawdown duration
    port_cum_series= pd.DataFrame(index = port_return.index[:], 
                                  columns=['CumulativeReturn','WaterMark','Drawdown','Dwn_Duration'])

    for i in range(len(port_return)):
        if i == 0:
            port_cum_series['CumulativeReturn'].iloc[i] = 0
            port_cum_series['WaterMark'].iloc[i] = port_cum_series['CumulativeReturn'].iloc[i]
        else:
            port_cum_series['CumulativeReturn'].iloc[i] = (1 + port_return['Values'].iloc[i]) * (1 + port_cum_series['CumulativeReturn'].iloc[i-1]) - 1
            if port_cum_series['CumulativeReturn'].iloc[i] > port_cum_series['WaterMark'].iloc[i-1]:
                port_cum_series['WaterMark'].iloc[i] = port_cum_series['CumulativeReturn'].iloc[i]
            else:
                port_cum_series['WaterMark'].iloc[i] = port_cum_series['WaterMark'].iloc[i-1]  

        port_cum_series['Drawdown'].iloc[i] = ((1 + port_cum_series['WaterMark'].iloc[i]) - (1 + port_cum_series['CumulativeReturn'].iloc[i])) / (1 + port_cum_series['WaterMark'].iloc[i])
        if i == 0:
            port_cum_series['Dwn_Duration'].iloc[i] = 0
        else:
            if port_cum_series['Drawdown'].iloc[i] != 0:
                port_cum_series['Dwn_Duration'].iloc[i] = port_cum_series['Dwn_Duration'].iloc[i-1] + 1
            else:
                port_cum_series['Dwn_Duration'].iloc[i] = 0 


    max_drawdown = port_cum_series['Drawdown'].max()
    max_dwn_duration = port_cum_series['Dwn_Duration'].max()
    total_return = port_value.iloc[-1]/port_value.iloc[0] - 1 
    
    #annual return
    ann_return = (1 + total_return) ** (1 / float((len(port_value)) / 250.0)) - 1 

    #sharpe ratio
    sharpe_ratio = (ann_return) / ann_vol

    print('annual volatility:{}'.format(ann_vol.iloc[0]))
    print('annual return:{}'.format(ann_return.iloc[0]))
    print('sharpe ratio:{}'.format(sharpe_ratio.iloc[0]))
    print('max drawdown:{}'.format(max_drawdown))
    print('max down duration:{} weeks'.format(max_dwn_duration))
    
def record_transaction(symbol, action_side, date, price, quantity, fee, transaction_history):
    '''
    action_side: long or close
    date: date for executing the action
    quantity: position size
    fee: one trip cost
    transaction_history: record all the transaction here
    '''

    info = {
                'Date': date,
                'Action': action_side,
                'Price': price,
                'Quantity': quantity,
                'Fee': fee
            }
    if symbol in transaction_history:
        transaction_history[symbol].append(info)
    else:
        transaction_history.update({symbol: [info]})
        
        
    return transaction_history

def record_holdings(symbol, starting_point, price, shares_change, fee, holdings, target = np.nan):
    '''
    starting_point: date for executing the action
    quantity: +/- position size
    fee: one trip cost
    holdings: record all the stock holdings info
    target: default is "nan", or you can specify the price level you prefer 
    '''
    if symbol in holdings:
        
        if math.isnan(target):
            target = holdings[symbol]['Target'] 
        
        info = {
                'Index': starting_point,
                'Price': price,
                'Quantity': holdings[symbol]['Quantity'] + shares_change,
                'Fee': fee,
                'Target': target
            }
        if info['Quantity'] == 0:
            del holdings[symbol]
        else:
            holdings[symbol].update(info)
    else:
        info = {
                'Index': starting_point,
                'Price': price,
                'Quantity': shares_change,
                'Fee': fee,
                'Target': target
            }
        holdings.update({symbol: info})
        
        
    return holdings

In [ ]:
ticker = ['NASDAQ' , 'SOX' , 'NYSE' , 'DJUSFN' , 'NBI' , 'AMEX' , 'IHX' , 'BCOMLC' , 'MXWO0EN' , 'NYE' , 'SIN' , 'DB1LUX' , 'BENCHMARK']###公司名稱
daily_data = pd.read_excel('Python index data-latest.xlsx', index_col = 0, header = [0, 1])###讀資料
daily_data.fillna(method = 'ffill', inplace = True)

